# Setup 

## ATTEN! - Not in venv /requirements yet, until we know we do Twitter

In [6]:
# Task specific
#from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

# The usuals
import pandas as pd
import numpy as np
import time
import json

# Unique
from html import unescape

# OS etc
from os import listdir
from os.path import isfile, join

In [3]:
# Paths
DATA_PATH = '../data/'
OUTPUT = '../data/raw/'

CREDS_PATH = '../credentials/'

In [65]:
# Filenames

creds_file = 'twitter_creds_v2.json'

In [101]:
# Get credentials

with open(CREDS_PATH + creds_file) as json_file:
    token_source = json.load(json_file)


key = token_source["API_key"]
secret = token_source["API_key_secret"]
token = token_source["Bearer_token"]
# consumer_key = token_source['consumer_key']
# consumer_secret =token_source['consumer_secret']
access_token = token_source['access_token']
access_token_secret = token_source['access_token_secret']
# bearer_token = token_source['bearer_token']

In [ ]:
# Reminder to export json -> will integrate and delete asap
      
line = {'id': d + '_' + in_file.split('.')[0], 
      'faces_mode': mode_faces_scene, 
      'faces_max': max_faces_scene, 
      'frames_w_face': frames_with_face_scene}

with open ('./data/video_faces.jsonl', 'a') as f:
      json.dump(line, f)
      f.write('\n')   

# Main Twitter classes (possibly outdated)

In [9]:
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
    
        self.twitter_user = twitter_user
    
    def get_twitter_client_api(self):
        return self.twitter_client
    
    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user, tweet_mode='extended').items(num_tweets):
            tweets.append(tweet)
        return tweets
    
    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list
    
    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


class TwitterAuthenticator():
    
    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

class TwitterStreamer():
    
    def __init__(self):
        self.twitter_authenicator = TwitterAuthenticator()
        
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        
        listener = TwitterListener(fetched_tweets_filename)
        auth =  self.twitter_authenicator.authenticate_twitter_app()
        stream =  Stream(auth, listener)

        stream.filter(track=hash_tag_list)    

# class TwitterListener(StreamListener):
    
#     def __init__(self, fetched_tweets_filename):
#         self.fetched_tweets_filename = fetched_tweets_filename
    
#     def on_data(self, data):
#         try:
#             print(data)
#             with open (self.fetched_tweets_filename, 'a') as tf:
#                 tf.write(data)
#             return True
#         except BaseException as e:
#             print('Error on data: %s' %str(e))
#         return True
    
#     def on_error(self, status):
#         if status == 420:
#             return False
#             # rate limit
#         print(status)
        
class TweetAnalyzer():
    def tweets_to_df(self, tweets):
        #tweets=unescape(tweet.full_text)

        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        return df


# V2 new search (lots to re-learn)

In [146]:
#import requests

client = tweepy.Client( bearer_token=token, 
                        consumer_key=key, 
                        consumer_secret=secret,
                        access_token=access_token,
                        access_token_secret=access_token_secret,
                        #return_type = requests.Response,
                        wait_on_rate_limit=True)

In [147]:
# Test

In [ ]:
# Define query
query = 'from:elonmusk -is:retweet'

# Get 10 tweets
tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id', 'created_at'],
                                    max_results=10)

In [154]:
for tweet in tweets.data:
    print(tweet)
    print('\n')

Oh hi lol


Berlin rocks


@Erdayastronaut @Tesla Raptor pump power is over 100MW per engine &amp; 33 engines, means over 3GW. Not even remotely possible for electric motors &amp; batteries to compete.


@GerberKawasaki We aim for 30% GM or ~10% profitability, all costs included


@PopMech Good summary


https://t.co/VlIZsJyH4F


@JuicyCreativ3 George Lucas once told me that the only reason Star Wars got made is because the old studio heads died


@JuicyCreativ3 Old people don’t change their minds, with rare exception, they just die. Without death, there would not be change.


I am inspired by curiosity. 

That is what drives me. 

So let us expand the scope &amp; scale of consciousness so that we may aspire to understand the Universe.


@WholeMarsBlog There will come a time in the future where people will be surprised that we let practically anyone drive a 2 ton death machine anywhere they want




## Start: main

In [73]:
# Define the search term and the date_since
search_words = "#Bucha -is:retweet"
date_since = "2022-04-02T00:00:00Z"

In [220]:
tweets = client.search_recent_tweets(query=search_words,
                                    expansions=['author_id','referenced_tweets.id','referenced_tweets.id.author_id','entities.mentions.username','attachments.media_keys','in_reply_to_user_id','geo.place_id'],
                                    tweet_fields=['author_id', 'created_at', 'geo', 'context_annotations', 'conversation_id', 'entities', 'in_reply_to_user_id', 'public_metrics', 'referenced_tweets'],
                                    media_fields=['alt_text','duration_ms','media_key','organic_metrics','preview_image_url','promoted_metrics','public_metrics','type','url'],
                                    place_fields=['place_type', 'geo', 'country'],
                                    user_fields=['profile_image_url', 'created_at', 'description', 'public_metrics', 'location', 'url', 'verified'],
                                    start_time=date_since,
                                    max_results=10)

In [223]:
# Get users list from the includes object
users = {u["id"]: u for u in tweets.includes['users']}

for tweet in tweets.data:
    if users[tweet.author_id]:
        user = users[tweet.author_id]
        print(user.name)

Roy Stewart. Слава Україні.
Alan Guerzoni
Naye
Militaer-fotos.de
Egdor Buf
Lallement Sébastien
Esperanza de una hora
David Chennell 🇪🇺🇺🇦🌱🍉 #FBPA #FBPE #FBPPR
OWIL
Rena Netjes


In [224]:
# Get list of places from includes object
places = {p["id"]: p for p in tweets.includes['places']}

for tweet in tweets.data:
    print(tweet.id)
    print(tweet.text)
    if places[tweet.geo['place_id']]:
        place = places[tweet.geo['place_id']]
        print(place.full_name)


KeyError: 'places'

In [225]:
tweets[1]

{'users': [<User id=1380877867580583946 name=Roy Stewart. Слава Україні. username=RoyStew78036358>,
  <User id=435239369 name=Промисловий Портал username=ua_industrial>,
  <User id=1413396592326062082 name=Alan Guerzoni username=AlanGuerzoni>,
  <User id=2687439506 name=stefano tartarotti username=tartaro7>,
  <User id=1011865740 name=Naye username=Elly_Rms>,
  <User id=1510184685388845056 name=Breaking News username=Breaking2022>,
  <User id=1191463975826939906 name=Militaer-fotos.de username=MSeehaase>,
  <User id=1430535474213425154 name=Alexandra von Stülpnagel username=a_v_stuelpnagel>,
  <User id=931594104659742720 name=Egdor Buf username=EgdorB>,
  <User id=1325039321301258240 name=Lallement Sébastien username=LallementSbast1>,
  <User id=2969361060 name=🇷🇺 botrytis Cinerea 🇫🇷 Французы за мир с Росси username=BotrytisC>,
  <User id=1134269070029852672 name=Esperanza de una hora username=EsperanzaHora>,
  <User id=1219583316 name=Embajada de Rusia en México username=EmbRusiaMexic

In [226]:
for tweet in tweets.data:
    print(tweet)

RT @ua_industrial: #Bucha. The territory of the children's camp.  Basement. All died with their hands tied. Everyone was tortured. https://…
RT @tartaro7: Le truppe russe si sono ritirate da #Bucha lasciandosi alle spalle fosse comuni e civili giustiziati in strada.
#stragi #guer…
RT @Breaking2022: President of Ukraine Volodymyr Zelensky visited #Bucha , #Irpen and #Stoyanka. https://t.co/dGb8b1WsP2
RT @a_v_stuelpnagel: Wie kann es zu solchen Gewaltexzessen wie in #Bucha kommen? Es wurde gemordet, vergewaltigt und geschändet. 

Wie könn…
Digitaler Chronist - #Bucha, Ukraine - Die Geschichte passt an vielen Stellen nicht
https://t.co/Nku8G0LJZ6 https://t.co/mVjMNw6Ona
RT @BotrytisC: Les soit disant "crimes de guerre" documenté par les Forces Armées #Ukrainienne à #Bucha et bien regardez cette vidéo ; Aprè…
RT @EmbRusiaMexico: ‼️ Recordamos la cronología de los hechos en #Bucha:

30 de marzo: retirada de la tropas rusas

31 de marzo: el alcalde…
RT @mi6rogue: #Bucha 
The dog stays with h

In [227]:
tweets[3]

{'newest_id': '1511032794532794373',
 'oldest_id': '1511032778804105226',
 'result_count': 10,
 'next_token': 'b26v89c19zqg8o3fpytlfnwmiiximc5l3zh3vkvaph27x'}

In [191]:
my_list = []

In [192]:
# Replace the limit=1000 with the maximum number of Tweets you want
for tweet in tweepy.Paginator(client.search_recent_tweets, query=search_words, 
                                    expansions=['author_id','referenced_tweets.id','referenced_tweets.id.author_id','entities.mentions.username','attachments.media_keys','in_reply_to_user_id','geo.place_id'],
                                    tweet_fields=['author_id', 'created_at', 'geo', 'context_annotations', 'conversation_id', 'entities', 'in_reply_to_user_id', 'public_metrics', 'referenced_tweets'],
                                    media_fields=['alt_text','duration_ms','media_key','organic_metrics','preview_image_url','promoted_metrics','public_metrics','type','url'],
                                    place_fields=['place_type', 'geo', 'country'],
                                    user_fields=['profile_image_url', 'created_at', 'description', 'public_metrics', 'location', 'url', 'verified'],
                                    start_time=date_since,
                                    #start_time=date_since,
                                    max_results=100).flatten(limit=200):
      print(tweet)
      my_list.append(tweet)

RT @PaulaChertok: ⚡️Zelensky visit #Bucha where Russian troops left behind civilian slaughter so great, a mass grave was dug to accommodate…
RT @BobRae48: The name of #VladimirPutin will be forever associated with crimes of cruelty and savagery. #Bucha, #Mariepol and countless ot…
RT @indyscotnews: As the #Bucha story does the media rounds it is worth reading #Telegram feeds of @EvaKBartlett and @realGonzaloLira – the…
@LAYS Song named "Lay's pay money to Putin. Russia kill Ukrainian"

Stop business in Russia!
#Lays #pepsico @LAYS @PepsiCo #bucha #WarCrimes https://t.co/EZin9J1sBw
RT @UKR_token: #Alemania expulsa a 40 diplomáticos rusos en respuesta a los asesinatos y las fosas comunes, encontradas en la ciudad de #Bu…
RT @Dpol_un: Looks like our UK colleagues seek to set the lowest possible standards of UNSC presidencies - our request has been rejected. T…
RT @GoncharenkoUa: This is #RussianWarCrimes in #Bucha 

In this car body of old lady without head. This is a real #GenocideOfUkra

In [196]:
for i in my_list:
      print(i)

RT @PaulaChertok: ⚡️Zelensky visit #Bucha where Russian troops left behind civilian slaughter so great, a mass grave was dug to accommodate…
RT @BobRae48: The name of #VladimirPutin will be forever associated with crimes of cruelty and savagery. #Bucha, #Mariepol and countless ot…
RT @indyscotnews: As the #Bucha story does the media rounds it is worth reading #Telegram feeds of @EvaKBartlett and @realGonzaloLira – the…
@LAYS Song named "Lay's pay money to Putin. Russia kill Ukrainian"

Stop business in Russia!
#Lays #pepsico @LAYS @PepsiCo #bucha #WarCrimes https://t.co/EZin9J1sBw
RT @UKR_token: #Alemania expulsa a 40 diplomáticos rusos en respuesta a los asesinatos y las fosas comunes, encontradas en la ciudad de #Bu…
RT @Dpol_un: Looks like our UK colleagues seek to set the lowest possible standards of UNSC presidencies - our request has been rejected. T…
RT @GoncharenkoUa: This is #RussianWarCrimes in #Bucha 

In this car body of old lady without head. This is a real #GenocideOfUkra

In [190]:
for page in search_results:
      # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
      # so we use expansions.flatten to get all the information in a single JSON
      result = expansions.flatten(page)
      for tweet in result:
            with open("results.jsonl", "w+" ) as f:
                  # Here we are writing 1 Tweet object JSON per line
                  f.write(json.dumps(tweet) + "\n")

In [ ]:
# Iterate and print tweets
for tweet in tweets:
    print(tweet.text)

# Some old starter code 

In [ ]:
# Activate API

twitter_client = TwitterClient()
tweet_analyzer = TweetAnalyzer()
api = twitter_cleint.get_twitter_client_api() 

In [78]:
tweets = api.user_timeline(screen_name='JPLehrke') #count=10
print(dir(tweets[1]))
test = api.user_timeline(screen_name='JPLehrke', count = 7, include_entities = True, tweet_mode='extended')
print(account.created_at)
    
print(tweets[6]._json)
print(tweets[6].entities)

df = tweet_analyzer.tweets_to_df(tweets)

BadRequest: 400 Bad Request
215 - Bad Authentication data.

In [32]:

deleted_accounts = []

accounts = []
created_list = []
favourite = []
friends = []
followers = []
location = []
status_count = []

for user in usernames:
    try:
        account = api.get_user(screen_name=user) #count=10
        #print(account._json)
        accounts.append(account._json)
        
        print(account.created_at)
        
        created_list.append(account.created_at)
        followers.append(account.followers_count)
        friends.append(account.friends_count)
        favourite.append(account.favourites_count)
        status_count.append(account.statuses_count)
        
        time.sleep(1)
    
    except:
        deleted_accounts.append(user)
        pass

NameError: name 'usernames' is not defined

In [ ]:
#usernames
deleted_accounts

In [ ]:
tweep = pd.DataFrame(list(zip(usernames, created_list, friends, followers, status_count)),
               columns =['user', 'created', 'friends', 'followers', 'status_count'])